In [1]:
import random
import time

def gen_alternant_code(F, n, m, k, supp, mult):
    """Генерация альтернантного кода с параметрами
    параметр F: расширение поля
    параметр n: длина кода
    параметр m: степень расширения
    параметр k: размерность лежащего в основе обобщённого кода Рида-Соломона (GRS)
    параметр supp: "support" вектор
    параметр mult: "multiplier" вектор
    выход: альтернантный код длины n и степени k
    """
    # === Вычисление обобщенного кода Рида-Соломона над F с размерностью k: === #
    # Построение образующей матрицы:
    G_GRS_k = matrix(F, k, n, 0)
    for i in range(k):
        for j in range(n):
            G_GRS_k[i,j] = mult[j]*supp[j]**i
    # Его дуальный код:
    GRS_k = LinearCode(G_GRS_k)
    GRS_k_dual = GRS_k.dual_code()
    print("FROM gen_altern_code, GRS_k_dual:", GRS_k_dual)

    # === Создание альтернантного кода над полем F_q
    # (подполе подкода GRS-кода): === #
    # Проверочная матрица:
    H_ = GRS_k_dual.parity_check_matrix()
    H = matrix(F.base_ring(), m*H_.nrows(), n, 0)
    print("FROM gen_altern_code, проверочная матрица H:")
    # Запись каждого входа H_ над F_q:
    FF = F.vector_space()
    for i in range(H_.nrows()):
        for j in range(n):
            elem = H_[i,j]
            coerced_elem = FF(elem)
            for l in range(m):
                H[l+i*m, j] = coerced_elem[l]
    print(H, end = '\n\n')
    
    # Построение порождающей матрицы G для альтернантного кода
    # из проверочной матрицы H (G*H^T=0):
    ker = H.right_kernel()
    print("FROM gen_altern_code, vector space of H:")
    print(ker, end = '\n\n')
    G_alternant = ker.basis_matrix()
    Alt_k = LinearCode(G_alternant)

    # === Проверка параметров: === #
    if Alt_k.dimension() == 0:
        raise ValueError("Неверные параметры!!!")

    # === Вернём альтернантный код === #
    return Alt_k


def gen_ecp(extension_field, n, k, supp, mult):
    """Генерация пары (А, В), исправляющей ошибки для альтернантного кода
    параметр extension_field: base_field лежащего в основе обобщённого кода Рида-Соломона (GRS)
    параметр n: длина кода
    параметр k: степень альтернантного кода
    параметр supp: support вектор альтернантного кода
    параметр mult: multiplier вектор альтернантного кода
    выход: пара (A, B), исправляющая ошибки
    """
    # Вычисление макс. кол-ва ошибок, которые могут быть добавлены к кодовому слову:
    t = floor(k/2)
    print("FROM gen_ecp: вычисление макс. кол-ва ошибок, которые можно добавить к кодовому слову:", t, end = '\n\n')

    # === Построение пары, исправляющей ошибки: === #
    # Порождающая матрица для A:
    G_A = matrix(extension_field, t+1, n, 0)
    for i in range(G_A.nrows()):
        for j in range(G_A.ncols()):
            G_A[i, j] = supp[j]**i
    print("FROM gen_ecp: порождающая матрица для А (т.е. G_A):")
    print(G_A, end = '\n\n')
    # Построение линейного кода A:
    A = LinearCode(G_A)
    print("FROM gen_ecp: линейный код А:")
    print(A, end = '\n\n')
    # Порождающая матрица для B:
    G_B = matrix(extension_field, t, n, 0)
    for i in range(G_B.nrows()):
        for j in range(G_B.ncols()):
            G_B[i,j] = mult[j]*(supp[j]**i)
    print("FROM gen_ecp: порождающая матрица для B (т.е. G_B):")
    print(G_B, end = '\n\n')
    #Построение линейного кода B:
    B = LinearCode(G_B)
    print("FROM gen_ecp: линейный код B:")
    print(B, end = '\n\n')

    # === Получение линейных кодов А и В: === #
    return A, B


def hide_structure(code):
    """Скрытие структуры кода путём изменения его порождающей матрицы G
    параметр code: код, структура которого должна быть скрыта
    выход: изменённая порождающая матрица, матрица перестановок P, шифрующая матрица S
    """
    # === Получение информации о коде: === #
    # Получение поля кода:
    K = code.base_field()
    print("FROM hide_structure: поле кода, т.е. К = ", K, end = '\n\n' )
    # Получение длины кода:
    n = code.generator_matrix().ncols()
    print("FROM hide_structure: длина кода n = ", n, end = '\n\n')
    # Получение размерности кода:
    dim_code = code.generator_matrix().nrows()
    print("FROM hide_structure: размерность кода dim = ", dim_code, end = '\n\n')

    # === Умножение порождающей матрицы на случайную перестановку
    # и случайную шифрующую матрицу: === #
    # Вычисление случайной матрицы перестановок P:
    indices = list(range(n))
    #indices = range(n)
    random.shuffle(indices)
    P = matrix(K, n, n, 0)
    for i in range(n):
        P[i, indices[i]] = K.one()
    print("FROM hide_structure: случайная матрица перестановок P:")
    print(P, end = '\n\n')

    # Вычисление случайной шифрующей матрицы S:
    S = random_matrix(K, dim_code, dim_code, algorithm='unimodular')
    print("FROM hide_structure: случайная шифрующая матрица S:")
    print(S, end = '\n\n')

    # === Вернём модифицированную порождающую матрицу G'=S*G*P === #
    print("FROM hide_structure: модифицированная порождающая матрица G'=S*G*P:")
    print(S*code.generator_matrix()*P, end = '\n\n')
    return S*code.generator_matrix()*P, P, S

    
def run_decryption(P, S, Alt_k, A, B, message): 
    """Дешифрование сообщения с заданными аргументами закрытого ключа"""

    # === Начало измерения времени: === #
    t0 = time.time()

    # === Процесс дешифрования: === #
    # Получение поля кода:
    base_field = Alt_k.base_field()
    print("FROM run_decryption: поле кода = ", base_field, end = '\n\n')
    
    # преобразование сообщения в вектор над базовым полем:
    word = vector(base_field, message)
    print("FROM run_decryption: преобразование сообщения в вектор над полем:")
    print(word, end = '\n\n')
    
    # Начало процесса дешифрования:
    code = decrypt(P, S, Alt_k, A, B, word)
    print("FROM run_decryption: процесс дешифрования :")
    print(code, end = '\n\n')
    
    # если процесс дешифрования завершился неудачно, вернём -1:
    if code == -1:
        return -1

    # === Остановка измерения времени: === #
    t1 = t1 = time.time() - t0
    print("На дешифрование затрачено:{0} sec.".format(t1))

    # === Вернём дешифрованное слово: === #
    return code

def decrypt(P, S, Alt_k, A, B, word): 
    """Дешифрование слова с заданными аргументами закрытого ключа"""

    # Умножить данное слово на P^{-1}:
    y = word * P.inverse()
    print("FROM decrypt: умножаем заданное слово на Р**(-1):")
    print(y, end = '\n\n')
    
    # Начать процесс декодирования:
    code_word = decode_using_ECP(Alt_k, A, B, y)
    print("FROM decrypt: процесс декодирования, вызов ф-ии decode_using_ECP:")
    print("code_word = ", code_word, end = '\n\n')
    
    # если процесс декодирования завершился неудачно, вернуть -1:
    if code_word==-1:
        return -1
    # Вычисление матрицы S*G:
    solve_mat = S*Alt_k.generator_matrix()
    print("FROM decrypt: вычисление матрицы S*G:")
    print(solve_mat, end = '\n\n')
    
    # Получение исходного сообщения:
    message = solve_mat.solve_left(code_word)
    print("FROM decrypt: исходное сообщение = ")
    print(message, end = '\n\n')
    

    # === Вернём исходное сообщение: === #
    return message


def decode_using_ECP(C, A, B, y):
    """Декодирование y = c + e, где c - элемент из C и e - вектор ошибок"""

    # === Начало измерения времени: === #
    t0 = time.time()

    # === Инициализирование: === #
    F = A.base_field()
    n = len(y)
    ky_element = 0
    K = F.base_ring()

    # === Вычисление "ядра" полученного слова y: === #
    ky_element = compute_kernel_element(A, B, y)
    print("FROM decode_using_ECP: вычисление 'ядра' полученного слова y: ")
    print(ky_element, end = '\n\n')
    
    # если вычисление "ядра" не удалось, вернуть -1:
    if ky_element == 0:
        return -1
    J = []
    for i in range(n):
        if ky_element[i] == 0:
            J.append(i)
    print("FROM decode_using_ECP: матрица J")
    print(J, end = '\n\n')

    # === Решение линейного уравнения, чтобы получить вектор ошибок: === #
    # Получение проверочной матрицы:
    H = C.parity_check_matrix()
    # Вычисление дополнения к J:
    J_comp = list(range(H.ncols()))
    for i in J:
        J_comp.remove(i)
    print("FROM decode_using_ECP: вычисление дополнения к матрице J")
    print(J_comp, end = '\n\n')
    
    # Построение матрицы, состоящей из J-индексированных столбцов H:
    H_J = H[:,J]
    # Построение матрицы, состоящей из J_comp-индексированных столбцов H:
    H_J_comp = H[:,J_comp]
    # Построение вектора, состоящего из J-индексированных вхождений y:
    y_J = vector(F,[y[i] for i in range(n) if i in J])
    # Построение вектора, состоящего из J_comp-индексированных вхождений y:
    y_J_comp = vector(F,[y[i] for i in range(n) if i not in J])
    # Вычисление пространства решений:
    temp = H_J*y_J.column()+H_J_comp*(y_J_comp.column())
    sol_ = H_J.solve_right(temp)
    sol_e = vector(K, n)
    # Вычисление вектора ошибки:
    index=0
    for j in J:
        sol_e[j] = sol_[index][0]
        index+=1
    print("FROM decode_using_ECP: вычисление вектора ошибки:")
    print(index, end = '\n\n')
    
    # Вычисление кодового слова как y-ошибку
    sol_c = y-sol_e
    print("FROM decode_using_ECP: вычисление кодового слова как y-ошибка:")
    print(sol_c, end = '\n\n')

    # === Остановка измерения времени: === #
    t1 = time.time() - t0
    print("Время, затраченное на процесс декодирования:{0} sec.".format(t1))

    # === Вернём декодированное кодовое слово: === #
    return sol_c


def compute_kernel_element(A, B, y):
    """Вычисление ядра ker_y для y и получение случайного элемента из ker_y"""

    # === Вычислим преобразующую матрицу: === #
    T = matrix(A.base_ring(), A.dimension(), B.dimension(), 0)
    for i in range(A.dimension()):
        for j in range(B.dimension()):
            T[i,j] = inner_star_prod(A.basis()[i], B.basis()[j], y)
    print("FROM compute_kernel_element: вычисление преобразующей матрицы (T)")
    print(T, end = '\n\n')
    
    # === Вычислим ядро и убедимся, что оно не пусто: === #
    ker = kernel(T)
    print("FROM compute_kernel_element: вычисление ядра матрицы T")
    print(ker, end = '\n\n')
    if ker.cardinality() == 0:
        raise ValueError("Пустое ядро!!!!")
    # === Вычисление случайного элемента из K_y: === #
    lambdas = ker.random_element()
    while lambdas == ker.zero():
        lambdas = ker.random_element()
    A_base_mat = matrix(A.base_ring(), A.basis())
    a_rand = lambdas * A_base_mat

    # === Вернём произвольный элемент из K_y: === #
    return a_rand

def inner_star_prod(a,b,y):
    """Вычисление скалярного произведения a * b и y"""

    # Получение длины вектора:
    n = len(y)
    # Вычисление <a*b,y>:
    res = 0
    for i in range(n):
        res += a[i]*b[i]*y[i]

    # === Вернём полученный элемент поля === #
    return res

def build_McEliece_cryptosystem(q, n, m, k):
    """ Параметры криптосистемы Мак-Элиса
    параметр q:   простое число, порядок
    параметр n:   длина линейного кода
    параметр m:   степень расширения
    параметр k:   размерность лежащего в основе обобщённого кода Рида-Соломона (GRS)
    выход:    открытый и закрытый ключ криптосистемы МакЭлиса
    """
    base_field = GF(q)
    extension_field = GF(q**m, 'z')

    # === Проверка параметров: === #
    # Длина кода должна быть меньше или равна количеству элементов поля:
    if n > extension_field.cardinality():
        raise ValueError("Неверные параметры: n > |F|")
    # Длина кода должна быть больше размера подпространства:
    if k > n:
        raise ValueError("Неверные параметры: k > n")
    print("Создание кода, где n={0}, k={1}, m={2} и q={3}".format(n, k, m, q))
    print()

    # === Метка времени начала работы === #
    t0 = time.time()

    # === Построение случайных "support" и "multiplier" векторов: === #
    # "Support": вектор, содержащий различные элементы поля расширения:
    a = extension_field.gen()
    support = [a^i for i in range(n-1)]+[0]
    print("support vector:", support, end = '\n\n')
    
    # "Multiplier": вектор, содержащий ненулевые элементы поля расширения:
    l = list(extension_field)
    l.remove(extension_field.zero())
    multiplier = []
    for i in range(n):
        multiplier.append(random.choice(l))
    print("multiplier, содержащий ненулевые элементы поля расширения: ", multiplier, end = '\n\n')

    # === Построение альтернантного кода: === #
    Alt_k = gen_alternant_code(extension_field, n, m, k, support, multiplier)
    print("Alt_k, альтернантный код: ", Alt_k, end = '\n\n')

    # === Построение пары, исправляющей ошибки === #
    A, B = gen_ecp(extension_field, n, k, support, multiplier)
    print("ECP (A, B): ", A, ";", B, end = '\n\n')

    # === Вычисление максимального количества позиций с ошибкой: === #
    t = floor(k/2)
    print("Максимальное кол-во позиций с ошибкой: ", t, end = '\n\n')

    # === Создание открытого ключа: === #
    G_public, P, S = hide_structure(Alt_k)
    print("Открытый ключ: ")
    print()
    print(hide_structure(Alt_k))

    # === Остановка измерения времени: === #
    t1 = time.time() - t0
    print("Размерность альтернантного кода={0}. Затрачено времени: {1} sec.".format(Alt_k.dimension(), t1))
    print()
    print("G_public: ")
    print(G_public, end = '\n\n')
    print("Случайная матрица перестановок P:")
    print(P, end = '\n\n')
    print("Случайная шифрующая матрица S:")
    print(S, end = '\n\n')
    print("Alt_k, альтернантный код: ", Alt_k, end = '\n\n')
    print("Matrix A:")
    print(A, end = '\n\n')
    print("Matrix B:")
    print(B, end = '\n\n')
    print("t = ", t, end = '\n\n')

    return G_public, P, S, Alt_k, A, B, t

